In [1]:
library(dplyr)
library(tidyr)
library(readr)
library(stringr)
library(ggplot2)
library(lubridate)
library(sf)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:dplyr’:

    intersect, setdiff, union


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Linking to GEOS 3.5.2, GDAL 2.4.2, PROJ 4.8.0



In [2]:
fp <- '/pool001/mfzhao/'

gec_to_iso <- read_csv(str_c(fp, 'geo_data/gec_to_iso.csv'))
as.data.frame(read_rds(str_c(fp, 'geo_data/europe_LRmap.RDS'))) %>%
    select(-geometry) -> eu_data
fips <- read_csv(str_c(fp, 'geo_data/county_fips_master.csv')) %>%
    mutate(fips = str_pad(fips, 5, pad = '0')) %>%
    select(fips, county_name, state_abbr, state_name) %>%
    distinct()

policy_eu <- read_csv(str_c(fp, 'policy/OxCGRT_latest.csv'))
policy_us <- read_csv(str_c(fp, 'policy/CUSP_policy.csv'))

countries <- data.frame(
    CountryCode = c('AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN',
                    'FRA', 'DEU', 'IRL', 'ITA', 'LVA', 'LIE', 'LTU', 'LUX', 'MLT',
                    'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE',
                    'CHE', 'GBR'), stringsAsFactors = F)

Parsed with column specification:
cols(
  Country = col_character(),
  gec = col_character(),
  iso2 = col_character(),
  iso3 = col_character()
)

Parsed with column specification:
cols(
  fips = col_double(),
  county_name = col_character(),
  state_abbr = col_character(),
  state_name = col_character(),
  long_name = col_character(),
  sumlev = col_double(),
  region = col_double(),
  division = col_double(),
  state = col_double(),
  county = col_double(),
  crosswalk = col_character(),
  region_name = col_character(),
  division_name = col_character()
)

Parsed with column specification:
cols(
  .default = col_double(),
  CountryName = col_character(),
  CountryCode = col_character(),
  M1_Wildcard = col_logical()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  State = col_character(),
  closeSchools = col_character(),
  closeDaycares = col_character(),
  closeNursing = col_character(),
  closeBusiness = col_character(),
  closeRestauran

In [3]:
policy_eu %>%
    inner_join(countries) %>%
    mutate(Date = ymd(Date)) %>%
    select(iso3 = CountryCode,
           date = Date,
           closeSchools   = `C1_School closing`,
           closeWork      = `C2_Workplace closing`,
           cancelEvents   = `C3_Cancel public events`,
           banGatherings  = `C4_Restrictions on gatherings`,
           closeTransport = `C5_Close public transport`,
           stayHome       = `C6_Stay at home requirements`,
           restrictMvmt   = `C7_Restrictions on internal movement`,
           travelBan      = `C8_International travel controls`,
           publicInfo     = `H1_Public information campaigns`) -> policy_eu
    

Joining, by = "CountryCode"



In [4]:
policy_eu %>%
    mutate(total = closeSchools + closeWork + cancelEvents + banGatherings + closeTransport + 
                   stayHome + restrictMvmt) %>%
    filter(total > 0) %>%
    group_by(iso3) %>%
    filter(row_number() == 1) %>%
    select(iso3, sdPolicy = date) -> sdPolicy_eu

policy_eu %>%
    filter(stayHome > 0) %>%
    group_by(iso3) %>%
    filter(row_number() == 1) %>%
    select(iso3, stayHome = date) -> stayHome_eu

policy_eu %>%
    group_by(iso3) %>%
    filter(closeWork == max(closeWork, na.rm = T)) %>%
    filter(row_number() == n()) %>%
    bind_rows(policy_eu %>%
              group_by(iso3) %>%
              filter(stayHome == max(stayHome, na.rm = T)) %>%
              filter(row_number() == n())) %>%
    arrange(iso3, date) %>%
    filter(row_number() == 1) %>%
    mutate(date = date + 1) %>%
    select(iso3, reopening = date) -> reopening_eu

sdPolicy_eu %>%
    left_join(stayHome_eu) %>%
    left_join(reopening_eu) %>%
    left_join(gec_to_iso) %>%
    ungroup() %>%
    mutate(iso2 = ifelse(iso2 == 'GB', 'UK', iso2)) %>%
    select(cluster = iso2, name = Country, sdPolicy, stayHome, reopening)  -> policyPeriods_eu

Joining, by = "iso3"

Joining, by = "iso3"

Joining, by = "iso3"



In [5]:
policy_us %>%
    rowwise() %>%
    mutate(sdPolicy = min(closeSchools, closeDaycares, closeNursing, closeRestaurants, 
                          closeGyms, closeMovies, stayHome, na.rm = T),
           reopening = min(endStayHome, reopenBusiness, na.rm = T)) %>%
    select(State, sdPolicy, stayHome, reopening) %>%
    mutate(sdPolicy  = mdy(sdPolicy),
           stayHome  = mdy(stayHome),
           reopening = mdy(reopening)) %>%
    inner_join(fips %>%
               mutate(cluster = str_sub(fips, 1, 2)) %>%
               select(State = state_name, cluster) %>%
               distinct()) %>%
    select(cluster, name = State, sdPolicy, stayHome, reopening) -> policyPeriods_us

Joining, by = "State"



In [6]:
bind_rows(policyPeriods_eu, policyPeriods_us) -> policyPeriods
write_csv(policyPeriods, str_c(fp, 'PROCESSED_DATA/policyPeriods.csv'))

In [10]:
eu_data %>%
    semi_join(policyPeriods_eu %>% select(CNTR_CODE = cluster) %>% distinct()) %>%
    select(key = id) %>%
    bind_rows(fips %>% select(key = fips)) -> keys

expand.grid(key = keys$key, 
            date = seq.Date(as.Date('2020-01-01'), as.Date('2020-06-30'), 'day'), 
            stringsAsFactors = F) %>%
    mutate(cluster = str_sub(key, 1, 2)) %>%
    left_join(policyPeriods %>%
              gather(key = period, value = date, sdPolicy, stayHome, reopening) %>%
              mutate(value = 1) %>%
              spread(key = period, value = value) %>%
              filter(!is.na(date)) %>%
              select(-name)) %>%
    arrange(key, date) %>%
    group_by(key) %>%
    fill(sdPolicy, stayHome, reopening) %>%
    replace_na(list(sdPolicy = 0, stayHome = 0, reopening = 0)) %>%
    mutate(sdPolicy = ifelse(stayHome == 1 | reopening == 1, 0, sdPolicy),
           stayHome = ifelse(reopening == 1, 0, stayHome)) -> policy

write_csv(policy, str_c(fp, 'PROCESSED_DATA/policy.csv'))

Joining, by = "CNTR_CODE"

Joining, by = c("date", "cluster")



In [7]:
policyPeriods

cluster,name,sdPolicy,stayHome,reopening
<chr>,<chr>,<date>,<date>,<date>
AT,Austria,2020-03-11,2020-03-16,2020-04-14
BE,Belgium,2020-03-10,2020-03-18,2020-05-11
BG,Bulgaria,2020-03-05,2020-03-18,2020-05-08
CH,Switzerland,2020-02-25,2020-03-17,2020-04-27
CY,Cyprus,2020-03-10,2020-03-24,2020-05-21
CZ,Czechia,2020-03-10,2020-03-15,2020-04-20
DE,Germany,2020-02-26,2020-03-09,2020-05-06
DK,Denmark,2020-03-03,2020-03-03,2020-06-19
ES,Spain,2020-03-09,2020-03-14,2020-05-22
